In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)

memory = ConversationBufferMemory(
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer the question using the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)

chain = (
    {   
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),
        "history": load_memory
    }
    | prompt 
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question) 
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

In [2]:
invoke_chain("Is Aaronson guilty?")

According to Winston's thoughts in the context, he believes that Jones, Aaronson, and Rutherford are guilty of the crimes they were charged with, despite having previously seen evidence that disproved their guilt. He has convinced himself that the photograph that disproved their guilt never existed and that he had invented it. Therefore, in the context of the Party's narrative, Aaronson is considered guilty.


In [3]:
invoke_chain("What message did he write in the table?")

Winston traced "2+2=5" in the dust on the table.


In [4]:
invoke_chain("Who is Julia?")

Julia is a character in the story who is Winston's love interest. She is a fellow Party member who initially appears to conform to the Party's rules but secretly rebels against it. Julia and Winston share a romantic relationship and engage in acts of rebellion against the oppressive regime of the Party. She represents a sense of individuality and resistance to the Party's control.


In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Is Aaronson guilty?'),
  AIMessage(content="According to Winston's thoughts in the context, he believes that Jones, Aaronson, and Rutherford are guilty of the crimes they were charged with, despite having previously seen evidence that disproved their guilt. He has convinced himself that the photograph that disproved their guilt never existed and that he had invented it. Therefore, in the context of the Party's narrative, Aaronson is considered guilty."),
  HumanMessage(content='What message did he write in the table?'),
  AIMessage(content='Winston traced "2+2=5" in the dust on the table.'),
  HumanMessage(content='Who is Julia?'),
  AIMessage(content="Julia is a character in the story who is Winston's love interest. She is a fellow Party member who initially appears to conform to the Party's rules but secretly rebels against it. Julia and Winston share a romantic relationship and engage in acts of rebellion against the oppressive regime of the Party.